<a href="https://colab.research.google.com/github/yishairasowsky/Sefaria-Project/blob/master/ML/topic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd

In [0]:
prayer_df = pd.read_csv('/content/prayer-sources.csv', 
                        encoding="latin-1"
                        # encoding="utf-8"
                        )
not_prayer_df = pd.read_csv('/content/not-prayer-sources.csv', 
# not_prayer_df = pd.read_csv('/content/not-prayer-sources-more.csv', 
                            encoding="latin-1"
                              # encoding="utf-8"
                            )

In [8]:
prayer_df.sample(3)

,Ref,En,He
880,Exodus 13:1,"And the LORD spoke unto Moses, saying:",×Ö·×Ö°×Ö·×Ö¼ÖµÖ¥×¨ ×Ö°××Ö¸Ö× ×Ö¶×Ö¾...
251,Sefer Chasidim 1007:1,NaN,×¨× ××× ××× ××××¨ ××××× ××ª××...
155,"Midrash Tanchuma, Vaetchanan 3:1",(Deut. 3:23:) âI besought the Lord.â This ...,"×Ö¼Ö¸×Ö¸×¨ ×Ö·×Öµ×¨, ×Ö¸×Ö¶×ªÖ°×Ö·× Ö¼Ö..."


In [9]:
prayer_df['is_prayer'] = 'yes'
prayer_df = prayer_df[['En','is_prayer']]

prayer_df.tail(3)

,En,is_prayer
1049,"It is true that you are the one who judges, an...",yes
1050,In these Berachot (blessings) we bow down: the...,yes
1051,On seeds he says [the Beracha (blessing)] Bore...,yes


In [10]:
not_prayer_df.sample(3)

,Ref,En,He,Topics
1040,Bava Kamma 2a:4,The common denominator of the components in al...,××¦× ××©×× ×©××× ×©××¨×× ×××××§...,laws-of-damages-from-an-ox-that-gored|oxen|law...
1035,Sefer Chasidim 246:1,NaN,×× ×¨×××ª ××× ×©××× ×× ×× ××ª×§×...,names|sons|places
391,"Sefat Emet, Numbers, Shavuot 3:5",NaN,"××""×¢ ××¢×¦×¨×ª ××¢×× × ×××. ×× ××...",eating


In [11]:
not_prayer_df['is_prayer'] = 'no'
not_prayer_df = not_prayer_df[['En','is_prayer']]
not_prayer_df.sample(3)

,En,is_prayer
955,MISHNA: In the case of one who deposits money ...,no
78,NaN,no
313,Samuel hinted to Saul about these mysteries wh...,no


In [12]:
all_prayer_frames = [prayer_df, not_prayer_df]
all_prayer_df = pd.concat(all_prayer_frames)
all_prayer_df.reset_index(inplace=True)
all_prayer_df=all_prayer_df.rename(columns = {'En':'english_text'})


# all_prayer_df.style.set_properties({'align': 'right'})

# all_prayer_df.tail(3)
all_prayer_df.sample(5)
# all_prayer_df.loc[[345]]

,index,english_text,is_prayer
273,273,"'××ª×§×¨× ×©× ×, the meaning of the words ...",yes
83,83,And Moses and Aaron went out from Pharaoh; and...,yes
1376,324,"And furthermore, if it should enter your mind ...",no
343,343,The Gemara relates another incident involving ...,yes
569,569,NaN,yes


In [13]:
del all_prayer_df['index']

all_prayer_df.sample(3)

,english_text,is_prayer
1980,NaN,no
217,NaN,yes
1342,MISHNA: All foods that became ritually impure ...,no


In [14]:
print("Before dropping duplicates",all_prayer_df.shape)
all_prayer_df.drop_duplicates(inplace=True)
print("After dropping duplicates",all_prayer_df.shape)

print("\nNulls before dropping:",all_prayer_df.isnull().sum()[0])
all_prayer_df.dropna(inplace=True)
print("Nulls after dropping:",all_prayer_df.isnull().sum()[0])

# all_prayer_df.isnull().sum()
# all_prayer_df.sample(5)

Before dropping duplicates (2103, 2)
After dropping duplicates (1223, 2)

Nulls before dropping: 2
Nulls after dropping: 0


In [15]:
shape = all_prayer_df.shape
print(f"{shape[0]} rows and {shape[1]} columns")

1221 rows and 2 columns


In [16]:
all_prayer_df['is_prayer'] = all_prayer_df['is_prayer'].map({'no': 0, 'yes': 1})
all_prayer_df.sample(5)

,english_text,is_prayer
1340,"The Jewish neighbors came and spoke to Abaye, ...",0
996,The Gemara asks: Is that to say that one who r...,1
281,"And furthermore, Rabbi Yosei said: May my port...",1
1638,"When Jehoiakim came along and reigned, he said...",0
15,On the topic of prayers recited while travelin...,1


In [17]:
print('Prayer:', (all_prayer_df.is_prayer.values == 1).sum())
print('Not prayer:', (all_prayer_df.is_prayer.values == 0).sum())

Prayer: 553
Not prayer: 668


In [18]:
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

# BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_ \b\d+\b]')
STOPWORDS = set(stopwords.words('english'))

def process_text(text):
    #  text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

# whitelist = set('abcdefghijklmnopqrstuvwxyz ZXCVBNMASDFGHJKLQWERTYUIOP')
# genres['movie_id'] = genres['movie_id'].astype(str)

# def process_text(text):
#     '''
#     1. Keep only letters and spaces
#     2. Remove stopwords
#     3. Return str of clean text words
#     '''
#     #1
#     chars_str = ''.join(filter(whitelist.__contains__, text))
#     #2
#     clean_words = [word.lower() for word in chars_str.split() if word.lower() not in stopwords.words('english')]
#     #3
#     return ' '.join(clean_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
sample_text = '''aBC.!@#$%^&    c*(בחתו ‎‎‎ או‎ בקרחתו — une...
233 the gns: ac ...
1041    7. a pien p'''
process_text(sample_text)

'abc # c une233 gns ac 1041 7 pien p'

In [20]:
all_prayer_df.sample(5)

,english_text,is_prayer
1730,××¢×©××ª ××¨ ××× ×××¡××¨×ª× AND THOU...,0
528,JOY: The second [branch of love of G-d] is joy...,1
1465,(1) Sins between man and G-d only. For example...,0
1066,"But if its name must be á¸¥agav , one might ha...",0
1375,And the women said unto Naomi: âBlessed be t...,0


In [21]:
processed_df = all_prayer_df
processed_df['english_text'] = processed_df['english_text'].apply(process_text)
processed_df.head(5)

,english_text,is_prayer
0,another abba binyamins statements regard laws ...,1
1,taught baraita abba binyamin said ones prayer ...,1
2,additionally ravin bar rav adda said rabbi yit...,1
3,similarly taught baraita rabbi yishmael ben el...,1
4,meaning written let prayer unto lord time favo...,1


In [22]:
# all_prayer_df['En'] = all_prayer_df['En'].apply(stemming)
# all_prayer_df.head()
stemmed_df = processed_df
stemmed_df['english_text'] = stemmed_df['english_text'].apply(stemming)
stemmed_df.head()

,english_text,is_prayer
0,anoth abba binyamin statement regard law praye...,1
1,taught baraita abba binyamin said one prayer f...,1
2,addit ravin bar rav adda said rabbi yitzak sai...,1
3,similar taught baraita rabbi yishmael ben elis...,1
4,mean written let prayer unto lord time favor g...,1


In [0]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# prayer_words = ' '.join(list(all_prayer_df[stemmed_df['is_prayer'] == 1]['En']))
# prayer_wc = WordCloud(width = 512,height = 512).generate(prayer_words)
# plt.figure(figsize = (10, 8), facecolor = 'k')
# plt.imshow(prayer_wc)
# plt.axis('off')
# plt.tight_layout(pad = 0)
# plt.show()

In [0]:
# not_prayer_words = ' '.join(list(all_prayer_df[all_prayer_df['is_prayer'] == 0]['En']))
# not_prayer_wc = WordCloud(width = 512,height = 512).generate(not_prayer_words)
# plt.figure(figsize = (10, 8), facecolor = 'k')
# plt.imshow(not_prayer_wc)
# plt.axis('off')
# plt.tight_layout(pad = 0)
# plt.show()

In [0]:
df = stemmed_df

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', oov_token="<OOV>", lower=True)
tokenizer.fit_on_texts(X_train['english_text'].values)
# tokenizer.fit_on_texts(df['english_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))



In [0]:
"""
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
"""

In [0]:
from keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(df['english_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


In [0]:
Y = pd.get_dummies(df['is_prayer']).values
print('Shape of label tensor:', Y.shape)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


In [0]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Dense, SpatialDropout1D
# from tensorflow.python.keras.layers.recurrent import LSTM
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
# model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
batch_size = 64

# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [0]:
# X = df.english_text
# y = df.is_prayer
# print(f"Totals: {X.shape} / {y.shape}")

# from sklearn.model_selection import train_test_split

# # train, test = train_test_split(movies, random_state=42, test_size=0.30, shuffle=True)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56, test_size=0.20, shuffle=True)

# print(f"\nTrain: {X_train.shape} / {y_train.shape}")

# print(f"\nTest: {X_test.shape} / {y_test.shape}")

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

history = model.fit(X_train, Y_train, 
                    epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [0]:
import matplotlib.pyplot as plt
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


In [0]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
# plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();


In [0]:
# # Convert a collection of text documents to a matrix of token counts

# from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

# # vectorizer = CountVectorizer(analyzer=process_text)
# # vectorizer = CountVectorizer(ngram_range=(2, 4))
# # vectorizer = HashingVectorizer(ngram_range=(2, 4),alternate_sign=False)
# vectorizer = TfidfVectorizer(
#     # strip_accents='unicode', 
#     # analyzer='word', 
#     ngram_range=(1,3), 
#     # norm='l2', 
#     # max_features = 15000
#     # max_features = 12000
#     max_features = 10000
#     # max_features = 8000
#     # max_features = 5000
#     )

# # vectorizer = TfidfVectorizer(ngram_range=(2, 4))
# # vectorizer.fit(X_train)
# # vectorizer.get_feature_names()


In [0]:
# print("Shape of document term matrix")

# # X_train_dtm = vectorizer.transform(X_train)
# X_train_dtm = vectorizer.fit_transform(X_train)
# print(f"Train: {X_train_dtm.shape}")

# # prayer_bag_of_words = vectorizer.fit_transform(all_prayer_df['En'])
# X_test_dtm = vectorizer.transform(X_test)
# print(f"Test: {X_test_dtm.shape}")


In [0]:

# print(vectorizer.get_feature_names())


In [0]:
# from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, ComplementNB

# classifier = MultinomialNB()
# # classifier = ComplementNB()

# # for Gaussian need array format
# # X_train = X_train.toarray()
# # X_test = X_test.toarray()
# # classifier = GaussianNB()
# # classifier = BernoulliNB()

# %time classifier.fit(X_train_dtm, y_train)
# y_pred_class = classifier.predict(X_test_dtm)
# from sklearn import metrics

# print(f"\nBreakdown:\n{y_test.value_counts()[0]} non-prayer (0), and\n{y_test.value_counts()[1]} prayer (1)")

# null_accuracy = y_test.value_counts().head(1) / len(y_test)
# print(f'\nNull accuracy: {null_accuracy[0]}')

# non_prayer_passages = y_test.value_counts()[0]
# prayer_passages = y_test.value_counts()[1]
# print('Manual null accuracy:',(prayer_passages / (prayer_passages + non_prayer_passages)))

# print('\nAccuracy:',round(metrics.accuracy_score(y_test, y_pred_class),2))
# print('Precision:',round(metrics.precision_score(y_test, y_pred_class),2))
# print('Recall:',round(metrics.recall_score(y_test, y_pred_class),2))
# print('F1 score:',round(metrics.f1_score(y_test, y_pred_class),2))

# print("\nConfusion Matrix:")
# print(metrics.confusion_matrix(y_test, y_pred_class))


In [0]:

# from sklearn.model_selection import cross_val_score
# # clf = svm.SVC(kernel='linear', C=1)
# scores = cross_val_score(classifier, X, y, cv=5)
# scores


In [0]:
# TP = X_test[y_pred_class + y_test == 2]
# TN = X_test[y_pred_class + y_test == 0]
# FP = X_test[y_pred_class > y_test]
# FN = X_test[y_pred_class < y_test]

# pd.options.display.max_colwidth = 100

# # pd.style.set_properties(**{'text-align': 'left'})

# # print("\n*** TP ***")
# # print(TP.sample(5))
# # print("\n*** TN ***")
# # print(TN.sample(5))
# print("*** False Positives: Actually NOT prayer, but misclassified as prayer ***\n")
# print(FP.sample(10))
# print("\n\n*** False Negatives: Actually prayer, but misclassified as NOT prayer ***\n")
# print(FN.sample(10))

In [0]:
# FP.to_csv('FP', sep='\t', encoding='utf-8')
# FN.to_csv('FN', sep='\t', encoding='utf-8')
